This notebook is an interim solution for creating tasks before we can do so via a nice GUI

In [ ]:
# Get us into the correct directory to import our fb_auth class
# %cd ../lwll_api
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
import uuid
from typing import Union
sys.path.insert(0, os.path.abspath('../lwll_api'))

In [ ]:
from dotenv import load_dotenv
load_dotenv(dotenv_path="../lwll_api/.envs_dev")

In [ ]:
# By doing this import, it instantiates a class which reads our appropriate firebase account creds so that
# we can import firebase_admin directly after and be authenticated to the database of interest
from lwll_api.classes.fb_auth import fb_store
from lwll_api.classes.s3_cls import s3_operator
from lwll_utils import add_budget_labels_to_task, SeedLabelGenerator

# Define the Dev, Staging, and Eval Tasks
This is a manual process where the base and adaptation datasets are chosen and a whitelist is defined.

In [ ]:
# Lookup tasks if need be
task_lookup = fb_store.collection('prod_Task').document('ad348ff9-f6c2-4b21-9bbc-34de6e1ffe10')
docs = task_lookup.get()
docs.to_dict()

In [ ]:
# Get a new uuid if needed for new tasks
uuid.uuid4()

*Note*: In the dev and staging tasks, xview has too many seeds to save them to firebase. As a work-around, we just don't include seed labels for xview. We have told performers not to request seed labels for this task. 

In [ ]:
dev_tasks = [
    {
        "task_id": "problem_test_video_classification",
        "problem_type": "video_classification",
        "base_dataset": "hmdb",
        "base_evaluation_metrics": ["accuracy", "top_5_accuracy", "roc_auc", "cross_entropy_logloss", 
                                          "brier_score", "weighted_accuracy"],
        "adaptation_dataset": "hmdb",
        "adaptation_evaluation_metrics": ["accuracy", "top_5_accuracy", "roc_auc", "cross_entropy_logloss", 
                                          "brier_score", "weighted_accuracy"],
        "whitelist": ["imagenet_1k", "Kinetics400", "Kinetics600", "Kinetics700", "UCF101", "ActivityNet", 
                      "Moments-in-Time", "Sports1M", "YouTube8M", "HACS", "HVU", "AVA", 
                  "AVA-kinetics", "charades"],
        "optional_sub_tasks": []
    }
    ,
    {
        "task_id": '9c103cc4-e2e1-4070-9877-c3b64a6f327f',
        "problem_type": "video_classification",
        "base_dataset": "hmdb",
        "adaptation_dataset": "ucf101",
        "base_evaluation_metrics": ["accuracy", "top_5_accuracy", "roc_auc", "cross_entropy_logloss", 
                                          "brier_score", "weighted_accuracy"],
        "adaptation_evaluation_metrics": ["accuracy", "top_5_accuracy", "roc_auc", "cross_entropy_logloss", 
                                          "brier_score", "weighted_accuracy"],
        "whitelist": ["imagenet_1k", "Kinetics400", "Kinetics600", "Kinetics700", "UCF101", "ActivityNet", 
                      "Moments-in-Time", "Sports1M", "YouTube8M", "HACS", "HVU", "AVA", 
                  "AVA-kinetics", "charades"],
        "optional_sub_tasks": []
    },
    {"task_id": '295ef5ef-1a11-4bc9-8048-0240e9ccd7a1',
        "problem_type": "video_classification",
        "base_dataset": "ucf101",
        "adaptation_dataset": "hmdb",
        "base_evaluation_metrics": ["accuracy", "top_5_accuracy", "roc_auc", "cross_entropy_logloss", 
                                          "brier_score", "weighted_accuracy"],
        "adaptation_evaluation_metrics": ["accuracy", "top_5_accuracy", "roc_auc", "cross_entropy_logloss", 
                                          "brier_score", "weighted_accuracy"],
        "whitelist": ["imagenet_1k", "Kinetics400", "Kinetics600", "Kinetics700", "UCF101", "ActivityNet", 
                      "Moments-in-Time", "Sports1M", "YouTube8M", "HACS", "HVU", "AVA", 
                  "AVA-kinetics", "charades"]
    },
    {
        "task_id": "problem_test_image_classification",
        "problem_type": "image_classification",
		"base_dataset": "mnist",
		"base_evaluation_metrics": ["accuracy", "top_5_accuracy", "roc_auc", "cross_entropy_logloss", 
                                          "brier_score", "weighted_accuracy"],
		"adaptation_dataset": "mnist",
		"adaptation_evaluation_metrics": ["accuracy", "top_5_accuracy", "roc_auc", "cross_entropy_logloss", 
                                          "brier_score", "weighted_accuracy"],
        "whitelist": ["imagenet_1k", "coco2014", "voc2009", "imagenet_bbox",
                     "domain_net-quickdraw", "domain_net-painting", "domain_net-infograph",
                     "domain_net-clipart", "domain_net-sketch", "cifar100", "domain_net-all",
                     "face_detection", "domain_net-real", "pool_car_detection", "google_open_image"],
        "optional_sub_tasks": []
    },
    {
        "task_id": "problem_test_obj_detection",
        "problem_type": "object_detection",
		"base_dataset": "face_detection",
		"base_evaluation_metrics": ["mAP"],
		"adaptation_dataset": "face_detection",
		"adaptation_evaluation_metrics": ["mAP"],
        "whitelist": ["imagenet_1k", "coco2014", "voc2009", "imagenet_bbox",
                     "domain_net-quickdraw", "domain_net-painting", "domain_net-infograph",
                     "domain_net-clipart", "domain_net-sketch", "cifar100", "mnist", 
                     "domain_net-real", "pool_car_detection", "google_open_image", "domain_net-all"],
        "optional_sub_tasks": []
    }
    ,
    {
        "task_id": '6d5e1f85-5d8f-4cc9-8184-299db03713f4',
        "problem_type": "image_classification",
		"base_dataset": "cifar100",
		"base_evaluation_metrics": ["accuracy", "top_5_accuracy", "roc_auc", "cross_entropy_logloss", 
                                          "brier_score", "weighted_accuracy"],
		"adaptation_dataset": "mnist",
		"adaptation_evaluation_metrics": ["accuracy", "top_5_accuracy", "roc_auc", "cross_entropy_logloss", 
                                          "brier_score", "weighted_accuracy"],
        "whitelist": ["imagenet_1k", "coco2014", "voc2009", "imagenet_bbox",
                     "domain_net-quickdraw", "domain_net-painting", "domain_net-infograph",
                     "domain_net-clipart", "domain_net-sketch", "domain_net-all",
                     "face_detection", "domain_net-real", "pool_car_detection", "google_open_image"],
        "optional_sub_tasks": ["base_zsl"]
    },
    {
        "task_id": 'bbfadb2c-c7c3-4596-b548-3dd01a6d1d2c',
        "problem_type": "image_classification",
		"base_dataset": "domain_net-clipart",
		"base_evaluation_metrics": ["accuracy", "top_5_accuracy", "roc_auc", "cross_entropy_logloss", 
                                          "brier_score", "weighted_accuracy"],
		"adaptation_dataset": "domain_net-sketch",
		"adaptation_evaluation_metrics": ["accuracy", "top_5_accuracy", "roc_auc", "cross_entropy_logloss", 
                                          "brier_score", "weighted_accuracy"],
        "whitelist": ["imagenet_1k", "coco2014", "voc2009", "imagenet_bbox",
                     "domain_net-quickdraw", "domain_net-painting", "domain_net-infograph",
                     "cifar100", "mnist",
                     "face_detection", "domain_net-real", "pool_car_detection", "google_open_image"],
        "optional_sub_tasks": ["base_zsl", "adaptation_zsl"]
    },
    {
        "task_id": 'b01a6738-0b85-46c2-9318-16c3e2ef0f6d',
        "problem_type": "image_classification",
		"base_dataset": "domain_net-real",
		"base_evaluation_metrics": ["accuracy", "top_5_accuracy", "roc_auc", "cross_entropy_logloss", 
                                          "brier_score", "weighted_accuracy"],
		"adaptation_dataset": "cifar100",
		"adaptation_evaluation_metrics": ["accuracy", "top_5_accuracy", "roc_auc", "cross_entropy_logloss", 
                                          "brier_score", "weighted_accuracy"],
        "whitelist": ["imagenet_1k", "coco2014", "voc2009", "imagenet_bbox",
                     "domain_net-quickdraw", "domain_net-painting", "domain_net-infograph",
                     "domain_net-clipart", "domain_net-sketch", "mnist",
                     "face_detection", "pool_car_detection", "google_open_image"],
        "optional_sub_tasks": ["base_zsl", "adaptation_zsl"]
    },
    {
        "task_id": '9c391ced-19ed-489a-9be0-ecf11fc4dcab',
        "problem_type": "image_classification",
        "base_dataset": "mars_surface_imgs",
        "adaptation_dataset": "msl_curiosity_imgs",
        "base_evaluation_metrics": ["accuracy", "top_5_accuracy", "roc_auc", "cross_entropy_logloss", 
                                          "brier_score", "weighted_accuracy"],
        "adaptation_evaluation_metrics": ["accuracy", "top_5_accuracy", "roc_auc", "cross_entropy_logloss", 
                                          "brier_score", "weighted_accuracy"],
        "whitelist": [
            "imagenet_1k", "coco2014", "voc2009", "imagenet_bbox", "cifar100",
             "domain_net-quickdraw", "domain_net-painting", "domain_net-infograph",
             "domain_net-clipart", "domain_net-sketch", "domain_net-real", "mnist", 
             "face_detection", "pool_car_detection", "google_open_image"
        ],
        "optional_sub_tasks": []
    },
    {
        "task_id": 'ce491694-a660-471c-917b-8f9772b6df37',
        "problem_type": "image_classification",
        "base_dataset": "quick_draw_dataset",
        "adaptation_dataset": "imagenet_sketch",
        "base_evaluation_metrics": ["accuracy", "top_5_accuracy", "roc_auc", "cross_entropy_logloss", 
                                          "brier_score", "weighted_accuracy"],
        "adaptation_evaluation_metrics": ["accuracy", "top_5_accuracy", "roc_auc", "cross_entropy_logloss", 
                                          "brier_score", "weighted_accuracy"],
        "whitelist": [
            "imagenet_1k", "coco2014", "voc2009", "imagenet_bbox", "cifar100",
             "domain_net-quickdraw", "domain_net-painting", "domain_net-infograph",
             "domain_net-clipart", "domain_net-sketch", "domain_net-real", "mnist", 
             "face_detection", "pool_car_detection", "google_open_image"
        ],
        "optional_sub_tasks": []
    }, 
    {
        "task_id": '5683ee24-bf06-455a-bd25-0942fbcd3b0a',
        "problem_type": "image_classification",
        "base_dataset": "iNaturalist",
        "adaptation_dataset": "feathersv1",
        "base_evaluation_metrics": ["accuracy", "top_5_accuracy", "roc_auc", "cross_entropy_logloss", 
                                          "brier_score", "weighted_accuracy"],
        "adaptation_evaluation_metrics": ["accuracy", "top_5_accuracy", "roc_auc", "cross_entropy_logloss", 
                                          "brier_score", "weighted_accuracy"],
        "whitelist": [
            "imagenet_1k", "coco2014", "voc2009", "imagenet_bbox", "cifar100",
             "domain_net-quickdraw", "domain_net-painting", "domain_net-infograph",
             "domain_net-clipart", "domain_net-sketch", "domain_net-real", "mnist", 
             "face_detection", "pool_car_detection", "google_open_image"
        ],
        "optional_sub_tasks": []
    }, 
    {
        "task_id": '2a7bb8ea-7c1f-45be-9685-c8afb2a08f23',
        "problem_type": "image_classification",
        "base_dataset": "FEIFace",
        "adaptation_dataset": "SheffieldFace",
        "base_evaluation_metrics": ["accuracy", "top_5_accuracy", "roc_auc", "cross_entropy_logloss", 
                                          "brier_score", "weighted_accuracy"],
        "adaptation_evaluation_metrics": ["accuracy", "top_5_accuracy", "roc_auc", "cross_entropy_logloss", 
                                          "brier_score", "weighted_accuracy"],
        "whitelist": [
            "imagenet_1k", "coco2014", "voc2009", "imagenet_bbox", "cifar100",
             "domain_net-quickdraw", "domain_net-painting", "domain_net-infograph",
             "domain_net-clipart", "domain_net-sketch", "domain_net-real", "mnist", 
             "face_detection", "pool_car_detection", "google_open_image", "imagenet_22k"
        ],
        "optional_sub_tasks": []
    }, 
    {
        "task_id": 'd204349c-01f4-4ae4-844f-811185bf5f33',
        "problem_type": "image_classification",
        "base_dataset": "chestXray",
        "adaptation_dataset": "covid_chestXray",
        "base_evaluation_metrics": ["accuracy", "top_5_accuracy", "roc_auc", "cross_entropy_logloss", 
                                          "brier_score", "weighted_accuracy"],
        "adaptation_evaluation_metrics": ["accuracy", "top_5_accuracy", "roc_auc", "cross_entropy_logloss", 
                                          "brier_score", "weighted_accuracy"],
        "whitelist": [
            "imagenet_1k", "coco2014", "voc2009", "imagenet_bbox", "cifar100",
             "domain_net-quickdraw", "domain_net-painting", "domain_net-infograph",
             "domain_net-clipart", "domain_net-sketch", "domain_net-real", "mnist", 
             "face_detection", "pool_car_detection", "google_open_image", "imagenet_22k"
        ],
        "optional_sub_tasks": []
    }, 
    {
        "task_id": '3b6361a9-bf44-44b9-931f-edb7a5fccd3d',
        "problem_type": "image_classification",
        "base_dataset": "AID",
        "adaptation_dataset": "UCMerced_LandUse",
        "base_evaluation_metrics": ["accuracy", "top_5_accuracy", "roc_auc", "cross_entropy_logloss", 
                                          "brier_score", "weighted_accuracy"],
        "adaptation_evaluation_metrics": ["accuracy", "top_5_accuracy", "roc_auc", "cross_entropy_logloss", 
                                          "brier_score", "weighted_accuracy"],
        "whitelist": [
            "imagenet_1k", "coco2014", "voc2009", "imagenet_bbox", "cifar100",
             "domain_net-quickdraw", "domain_net-painting", "domain_net-infograph",
             "domain_net-clipart", "domain_net-sketch", "domain_net-real", "mnist", 
             "face_detection", "pool_car_detection", "google_open_image", "imagenet_22k"
        ],
        "optional_sub_tasks": []
    }, 
            ############################################
    {
        "task_id": "4d924004-347e-4043-8dd2-f4f8b0f52efd",
        "problem_type": "object_detection",
		"base_dataset": "coco2014_mini",
		"base_evaluation_metrics": ["mAP"],
		"adaptation_dataset": "voc2009",
		"adaptation_evaluation_metrics": ["mAP"],
        "whitelist": ["imagenet_1k", "imagenet_bbox",
                     "domain_net-quickdraw", "domain_net-painting", "domain_net-infograph",
                     "domain_net-clipart", "domain_net-sketch", "cifar100", "mnist", "domain_net-all",
                     "face_detection", "domain_net-real", "pool_car_detection", "google_open_image"],
        "optional_sub_tasks": []
    },
    {
        "task_id": "d48f8a99-ba12-4df8-a74a-d06413b0f1ba",
        "problem_type": "object_detection",
		"base_dataset": "voc2009",
		"base_evaluation_metrics": ["mAP"],
		"adaptation_dataset": "pool_car_detection",
		"adaptation_evaluation_metrics": ["mAP"],
        "whitelist": ["imagenet_1k", "coco2014", "imagenet_bbox",
                     "domain_net-quickdraw", "domain_net-painting", "domain_net-infograph",
                     "domain_net-clipart", "domain_net-sketch", "cifar100", "mnist", "domain_net-all",
                     "face_detection", "domain_net-real", "google_open_image"],
        "optional_sub_tasks": []
    }
    ,
    {
        "task_id": "7c103ece-fd8e-483b-bede-b55e5ea57fe9",
        "problem_type": "object_detection",
		"base_dataset": "voc2009",
		"base_evaluation_metrics": ["mAP"],
		"adaptation_dataset": "face_detection",
		"adaptation_evaluation_metrics": ["mAP"],
         "whitelist": ["imagenet_1k", "coco2014", "imagenet_bbox",
                     "domain_net-quickdraw", "domain_net-painting", "domain_net-infograph",
                     "domain_net-clipart", "domain_net-sketch", "cifar100", "mnist", "domain_net-all",
                     "domain_net-real", "pool_car_detection", "google_open_image"],
        "optional_sub_tasks": []
    }
    ,
    {
        "task_id": "923cbe9c-1e4f-4a05-a156-dc972ef5edf5",
        "problem_type": "object_detection",
		"base_dataset": "pool_car_detection",
		"base_evaluation_metrics": ["mAP"],
		"adaptation_dataset": "xview",
		"adaptation_evaluation_metrics": ["mAP"],
         "whitelist": ["imagenet_1k", "coco2014", "imagenet_bbox", "voc2009",
                     "domain_net-quickdraw", "domain_net-painting", "domain_net-infograph",
                     "domain_net-clipart", "domain_net-sketch", "cifar100", "mnist", "domain_net-all",
                     "domain_net-real", "google_open_image"]
    }
    ,
    {
        "task_id": 'dc75cc32-5db1-4767-b41f-b3dfa6b086a9',
        "problem_type": "object_detection",
        "base_dataset": "deep_fashion",
        "adaptation_dataset": "deep_fashion_2",
        "base_evaluation_metrics": ["mAP"],
        "adaptation_evaluation_metrics": ["mAP"],
        "whitelist": ["imagenet_1k", "coco2014", "imagenet_bbox", "voc2009", "pool_car_detection", "xview",
                     "domain_net-quickdraw", "domain_net-painting", "domain_net-infograph",
                     "domain_net-clipart", "domain_net-sketch", "cifar100", "mnist", "domain_net-all",
                     "domain_net-real", "google_open_image"],
        "optional_sub_tasks": []
    },
    {
        "task_id": 'a1182934-69d2-4a46-bcbb-feab912cce2c',
        "problem_type": "object_detection", 
        "base_dataset": "vis_drone",
        "adaptation_dataset": "stanford_campus_dataset",
        "base_evaluation_metrics": ["mAP"],
        "adaptation_evaluation_metrics": ["mAP"],
        "whitelist": ["imagenet_1k", "coco2014", "imagenet_bbox", "voc2009", "pool_car_detection", "xview",
                     "domain_net-quickdraw", "domain_net-painting", "domain_net-infograph",
                     "domain_net-clipart", "domain_net-sketch", "cifar100", "mnist", "domain_net-all",
                     "domain_net-real", "google_open_image", "deep_fashion", "deep_fashion_2"],
        "optional_sub_tasks": []
    }
    ,
    {
        "task_id": '7589909f-ca68-4289-8624-25c62c49f0c6',
        "problem_type": "object_detection",
        "base_dataset": "widerperson",
        "adaptation_dataset": "nightowls",
        "base_evaluation_metrics": ["mAP"],
        "adaptation_evaluation_metrics": ["mAP"],
        "whitelist": ["imagenet_1k", "coco2014", "imagenet_bbox", "voc2009", "pool_car_detection", "xview",
             "domain_net-quickdraw", "domain_net-painting", "domain_net-infograph",
             "domain_net-clipart", "domain_net-sketch", "cifar100", "mnist", "domain_net-all",
             "domain_net-real", "google_open_image", "deep_fashion", "deep_fashion_2", "vis_drone", "stanford_campus_dataset"],
        "optional_sub_tasks": []
    },
    {
    "task_id": '7e360ff2-291f-4bbc-b2c5-8a484f04e035',
    "problem_type": "image_classification",
    "base_dataset": "domain_net-painting",
    "base_evaluation_metrics": ["accuracy", "top_5_accuracy", "roc_auc", "cross_entropy_logloss", "brier_score", "weighted_accuracy"],
    "adaptation_dataset": "domain_net-real",
    "adaptation_evaluation_metrics": ["accuracy", "top_5_accuracy", "roc_auc", "cross_entropy_logloss", "brier_score", "weighted_accuracy"],
    "whitelist": ["imagenet_1k", "coco2014", "voc2009", "imagenet_bbox",
                 "domain_net-quickdraw", "domain_net-infograph",
                 "domain_net-clipart", "domain_net-sketch", "cifar100",
                 "face_detection", "pool_car_detection", "google_open_image", 
                  "mars_surface_imgs", "msl_curiosity_imgs", "deep_fashion", "deep_fashion_2", "xview", 
                  "stanford_campus_dataset", "vis_drone"],
    "optional_sub_tasks": []
    },
    {
        "task_id": '10c7919a-a8e3-440d-9bf9-959d4f1f880c',
        "problem_type": "object_detection", 
        "base_dataset": "stanford_campus_dataset",
        "adaptation_dataset": "pool_car_detection",
        "base_evaluation_metrics": ["mAP"],
        "adaptation_evaluation_metrics": ["mAP"],
        "whitelist": ["imagenet_1k", "coco2014", "voc2009", "imagenet_bbox",
                     "domain_net-quickdraw", "domain_net-painting", "domain_net-infograph",
                     "domain_net-clipart", "domain_net-sketch", "cifar100", "domain_net-all",
                     "face_detection", "domain_net-real", "google_open_image",
                      "mars_surface_imgs", "msl_curiosity_imgs", "deep_fashion", "deep_fashion_2", 
                      "xview"],
        "optional_sub_tasks": []
    }
    ,
    {
        "task_id": "problem_test_machine_translation",
        "problem_type": "machine_translation",
        "base_dataset": "wikimatrix-fas",
        "adaptation_dataset": "wikimatrix-fas",
        "base_evaluation_metrics": ["bleu"],
        "adaptation_evaluation_metrics": ["bleu"],
        "whitelist": []
    }
    ,
    {
        "task_id": '06023f86-a66b-4b2c-8b8b-951f5edd0f22',
        "problem_type": "machine_translation",
		"base_dataset": "global_voices",
		"base_evaluation_metrics": ["bleu"],
		"adaptation_dataset": "ted_talks",
		"adaptation_evaluation_metrics": ["bleu"],
       "whitelist": []
    }
    ,
    {
        "task_id": '3159d8ef-dfef-48fc-9325-829258be5f28',
        "problem_type": "machine_translation",
        "base_dataset": "cc_aligned_sinhala",
        "base_evaluation_metrics": ["bleu"],
        "adaptation_dataset": "wikimatrix-sin",
        "adaptation_evaluation_metrics": ["bleu"],
       "whitelist": []
    }
    ,
    {
        "task_id": '9877df6a-746f-400e-ac7a-71ca87f50057',
        "problem_type": "machine_translation",
        "base_dataset": "cc_aligned_polish",
        "base_evaluation_metrics": ["bleu"],
        "adaptation_dataset": "wikimatrix-pol",
        "adaptation_evaluation_metrics": ["bleu"],
       "whitelist": []
    }
    ,
    {
        "task_id": '36ff86c2-9403-4fbc-af78-ba2fb54c1734',
        "problem_type": "machine_translation",
        "base_dataset": "cc_aligned_persian",
        "base_evaluation_metrics": ["bleu"],
        "adaptation_dataset": "wikimatrix-fas",
        "adaptation_evaluation_metrics": ["bleu"],
       "whitelist": []
    }
    
]


In [ ]:
staging_tasks = [
    {
        "task_id": "problem_test_video_classification",
        "problem_type": "video_classification",
        "base_dataset": "hmdb",
        "base_evaluation_metrics": ["accuracy", "top_5_accuracy", "roc_auc", "cross_entropy_logloss", 
                                          "brier_score", "weighted_accuracy"],
        "adaptation_dataset": "hmdb",
        "adaptation_evaluation_metrics": ["accuracy", "top_5_accuracy", "roc_auc", "cross_entropy_logloss", 
                                          "brier_score", "weighted_accuracy"],
        "whitelist": ["imagenet_1k", "Kinetics400", "Kinetics600", "Kinetics700", "UCF101", "ActivityNet", 
                      "Moments-in-Time", "Sports1M", "YouTube8M", "HACS", "HVU", "AVA", 
                  "AVA-kinetics", "charades"],
        "optional_sub_tasks": []
    }
    ,
    {
        "task_id": '9c103cc4-e2e1-4070-9877-c3b64a6f327f',
        "problem_type": "video_classification",
        "base_dataset": "hmdb",
        "adaptation_dataset": "ucf101",
        "base_evaluation_metrics": ["accuracy", "top_5_accuracy", "roc_auc", "cross_entropy_logloss", 
                                          "brier_score", "weighted_accuracy"],
        "adaptation_evaluation_metrics": ["accuracy", "top_5_accuracy", "roc_auc", "cross_entropy_logloss", 
                                          "brier_score", "weighted_accuracy"],
        "whitelist": ["imagenet_1k", "Kinetics400", "Kinetics600", "Kinetics700", "UCF101", "ActivityNet", 
                      "Moments-in-Time", "Sports1M", "YouTube8M", "HACS", "HVU", "AVA", 
                  "AVA-kinetics", "charades"],
        "optional_sub_tasks": []
    },
    {"task_id": '295ef5ef-1a11-4bc9-8048-0240e9ccd7a1',
        "problem_type": "video_classification",
        "base_dataset": "ucf101",
        "adaptation_dataset": "hmdb",
        "base_evaluation_metrics": ["accuracy", "top_5_accuracy", "roc_auc", "cross_entropy_logloss", 
                                          "brier_score", "weighted_accuracy"],
        "adaptation_evaluation_metrics": ["accuracy", "top_5_accuracy", "roc_auc", "cross_entropy_logloss", 
                                          "brier_score", "weighted_accuracy"],
        "whitelist": ["imagenet_1k", "Kinetics400", "Kinetics600", "Kinetics700", "UCF101", "ActivityNet", 
                      "Moments-in-Time", "Sports1M", "YouTube8M", "HACS", "HVU", "AVA", 
                  "AVA-kinetics", "charades"]
    },
    {
        "task_id": "problem_test_image_classification",
        "problem_type": "image_classification",
		"base_dataset": "mnist",
		"base_evaluation_metrics": ["accuracy", "top_5_accuracy", "roc_auc", "cross_entropy_logloss", 
                                          "brier_score", "weighted_accuracy"],
		"adaptation_dataset": "mnist",
		"adaptation_evaluation_metrics": ["accuracy", "top_5_accuracy", "roc_auc", "cross_entropy_logloss", 
                                          "brier_score", "weighted_accuracy"],
        "whitelist": ["imagenet_1k", "coco2014", "voc2009", "imagenet_bbox",
                     "domain_net-quickdraw", "domain_net-painting", "domain_net-infograph",
                     "domain_net-clipart", "domain_net-sketch", "cifar100", "domain_net-all",
                     "face_detection", "domain_net-real", "pool_car_detection", "google_open_image"],
        "optional_sub_tasks": []
    },
    {
        "task_id": "problem_test_obj_detection",
        "problem_type": "object_detection",
		"base_dataset": "face_detection",
		"base_evaluation_metrics": ["mAP"],
		"adaptation_dataset": "face_detection",
		"adaptation_evaluation_metrics": ["mAP"],
        "whitelist": ["imagenet_1k", "coco2014", "voc2009", "imagenet_bbox",
                     "domain_net-quickdraw", "domain_net-painting", "domain_net-infograph",
                     "domain_net-clipart", "domain_net-sketch", "cifar100", "mnist", 
                     "domain_net-real", "pool_car_detection", "google_open_image", "domain_net-all"],
        "optional_sub_tasks": []
    }
    ,
    {
        "task_id": '6d5e1f85-5d8f-4cc9-8184-299db03713f4',
        "problem_type": "image_classification",
		"base_dataset": "cifar100",
		"base_evaluation_metrics": ["accuracy", "top_5_accuracy", "roc_auc", "cross_entropy_logloss", 
                                          "brier_score", "weighted_accuracy"],
		"adaptation_dataset": "mnist",
		"adaptation_evaluation_metrics": ["accuracy", "top_5_accuracy", "roc_auc", "cross_entropy_logloss", 
                                          "brier_score", "weighted_accuracy"],
        "whitelist": ["imagenet_1k", "coco2014", "voc2009", "imagenet_bbox",
                     "domain_net-quickdraw", "domain_net-painting", "domain_net-infograph",
                     "domain_net-clipart", "domain_net-sketch", "domain_net-all",
                     "face_detection", "domain_net-real", "pool_car_detection", "google_open_image"],
        "optional_sub_tasks": ["base_zsl"]
    },
    {
        "task_id": 'bbfadb2c-c7c3-4596-b548-3dd01a6d1d2c',
        "problem_type": "image_classification",
		"base_dataset": "domain_net-clipart",
		"base_evaluation_metrics": ["accuracy", "top_5_accuracy", "roc_auc", "cross_entropy_logloss", 
                                          "brier_score", "weighted_accuracy"],
		"adaptation_dataset": "domain_net-sketch",
		"adaptation_evaluation_metrics": ["accuracy", "top_5_accuracy", "roc_auc", "cross_entropy_logloss", 
                                          "brier_score", "weighted_accuracy"],
        "whitelist": ["imagenet_1k", "coco2014", "voc2009", "imagenet_bbox",
                     "domain_net-quickdraw", "domain_net-painting", "domain_net-infograph",
                     "cifar100", "mnist",
                     "face_detection", "domain_net-real", "pool_car_detection", "google_open_image"],
        "optional_sub_tasks": ["base_zsl", "adaptation_zsl"]
    },
    {
        "task_id": 'b01a6738-0b85-46c2-9318-16c3e2ef0f6d',
        "problem_type": "image_classification",
		"base_dataset": "domain_net-real",
		"base_evaluation_metrics": ["accuracy", "top_5_accuracy", "roc_auc", "cross_entropy_logloss", 
                                          "brier_score", "weighted_accuracy"],
		"adaptation_dataset": "cifar100",
		"adaptation_evaluation_metrics": ["accuracy", "top_5_accuracy", "roc_auc", "cross_entropy_logloss", 
                                          "brier_score", "weighted_accuracy"],
        "whitelist": ["imagenet_1k", "coco2014", "voc2009", "imagenet_bbox",
                     "domain_net-quickdraw", "domain_net-painting", "domain_net-infograph",
                     "domain_net-clipart", "domain_net-sketch", "mnist",
                     "face_detection", "pool_car_detection", "google_open_image"],
        "optional_sub_tasks": ["base_zsl", "adaptation_zsl"]
    },
    {
        "task_id": '9c391ced-19ed-489a-9be0-ecf11fc4dcab',
        "problem_type": "image_classification",
        "base_dataset": "mars_surface_imgs",
        "adaptation_dataset": "msl_curiosity_imgs",
        "base_evaluation_metrics": ["accuracy", "top_5_accuracy", "roc_auc", "cross_entropy_logloss", 
                                          "brier_score", "weighted_accuracy"],
        "adaptation_evaluation_metrics": ["accuracy", "top_5_accuracy", "roc_auc", "cross_entropy_logloss", 
                                          "brier_score", "weighted_accuracy"],
        "whitelist": [
            "imagenet_1k", "coco2014", "voc2009", "imagenet_bbox", "cifar100",
             "domain_net-quickdraw", "domain_net-painting", "domain_net-infograph",
             "domain_net-clipart", "domain_net-sketch", "domain_net-real", "mnist", 
             "face_detection", "pool_car_detection", "google_open_image"
        ],
        "optional_sub_tasks": []
    },
    {
        "task_id": 'ce491694-a660-471c-917b-8f9772b6df37',
        "problem_type": "image_classification",
        "base_dataset": "quick_draw_dataset",
        "adaptation_dataset": "imagenet_sketch",
        "base_evaluation_metrics": ["accuracy", "top_5_accuracy", "roc_auc", "cross_entropy_logloss", 
                                          "brier_score", "weighted_accuracy"],
        "adaptation_evaluation_metrics": ["accuracy", "top_5_accuracy", "roc_auc", "cross_entropy_logloss", 
                                          "brier_score", "weighted_accuracy"],
        "whitelist": [
            "imagenet_1k", "coco2014", "voc2009", "imagenet_bbox", "cifar100",
             "domain_net-quickdraw", "domain_net-painting", "domain_net-infograph",
             "domain_net-clipart", "domain_net-sketch", "domain_net-real", "mnist", 
             "face_detection", "pool_car_detection", "google_open_image"
        ],
        "optional_sub_tasks": []
    }, 
    {
        "task_id": '5683ee24-bf06-455a-bd25-0942fbcd3b0a',
        "problem_type": "image_classification",
        "base_dataset": "iNaturalist",
        "adaptation_dataset": "feathersv1",
        "base_evaluation_metrics": ["accuracy", "top_5_accuracy", "roc_auc", "cross_entropy_logloss", 
                                          "brier_score", "weighted_accuracy"],
        "adaptation_evaluation_metrics": ["accuracy", "top_5_accuracy", "roc_auc", "cross_entropy_logloss", 
                                          "brier_score", "weighted_accuracy"],
        "whitelist": [
            "imagenet_1k", "coco2014", "voc2009", "imagenet_bbox", "cifar100",
             "domain_net-quickdraw", "domain_net-painting", "domain_net-infograph",
             "domain_net-clipart", "domain_net-sketch", "domain_net-real", "mnist", 
             "face_detection", "pool_car_detection", "google_open_image"
        ],
        "optional_sub_tasks": []
    }, 
            #############################################
    {
        "task_id": "4d924004-347e-4043-8dd2-f4f8b0f52efd",
        "problem_type": "object_detection",
		"base_dataset": "coco2014_mini",
		"base_evaluation_metrics": ["mAP"],
		"adaptation_dataset": "voc2009",
		"adaptation_evaluation_metrics": ["mAP"],
        "whitelist": ["imagenet_1k", "imagenet_bbox",
                     "domain_net-quickdraw", "domain_net-painting", "domain_net-infograph",
                     "domain_net-clipart", "domain_net-sketch", "cifar100", "mnist", "domain_net-all",
                     "face_detection", "domain_net-real", "pool_car_detection", "google_open_image"],
        "optional_sub_tasks": []
    },
    {
        "task_id": "d48f8a99-ba12-4df8-a74a-d06413b0f1ba",
        "problem_type": "object_detection",
		"base_dataset": "voc2009",
		"base_evaluation_metrics": ["mAP"],
		"adaptation_dataset": "pool_car_detection",
		"adaptation_evaluation_metrics": ["mAP"],
        "whitelist": ["imagenet_1k", "coco2014", "imagenet_bbox",
                     "domain_net-quickdraw", "domain_net-painting", "domain_net-infograph",
                     "domain_net-clipart", "domain_net-sketch", "cifar100", "mnist", "domain_net-all",
                     "face_detection", "domain_net-real", "google_open_image"],
        "optional_sub_tasks": []
    }
    ,
    {
        "task_id": "7c103ece-fd8e-483b-bede-b55e5ea57fe9",
        "problem_type": "object_detection",
		"base_dataset": "voc2009",
		"base_evaluation_metrics": ["mAP"],
		"adaptation_dataset": "face_detection",
		"adaptation_evaluation_metrics": ["mAP"],
         "whitelist": ["imagenet_1k", "coco2014", "imagenet_bbox",
                     "domain_net-quickdraw", "domain_net-painting", "domain_net-infograph",
                     "domain_net-clipart", "domain_net-sketch", "cifar100", "mnist", "domain_net-all",
                     "domain_net-real", "pool_car_detection", "google_open_image"],
        "optional_sub_tasks": []
    }
    ,
    {
        "task_id": "923cbe9c-1e4f-4a05-a156-dc972ef5edf5",
        "problem_type": "object_detection",
		"base_dataset": "pool_car_detection",
		"base_evaluation_metrics": ["mAP"],
		"adaptation_dataset": "xview",
		"adaptation_evaluation_metrics": ["mAP"],
         "whitelist": ["imagenet_1k", "coco2014", "imagenet_bbox", "voc2009",
                     "domain_net-quickdraw", "domain_net-painting", "domain_net-infograph",
                     "domain_net-clipart", "domain_net-sketch", "cifar100", "mnist", "domain_net-all",
                     "domain_net-real", "google_open_image"]
    },
    {
        "task_id": 'dc75cc32-5db1-4767-b41f-b3dfa6b086a9',
        "problem_type": "object_detection",
        "base_dataset": "deep_fashion",
        "adaptation_dataset": "deep_fashion_2",
        "base_evaluation_metrics": ["mAP"],
        "adaptation_evaluation_metrics": ["mAP"],
        "whitelist": ["imagenet_1k", "coco2014", "imagenet_bbox", "voc2009", "pool_car_detection", "xview",
                     "domain_net-quickdraw", "domain_net-painting", "domain_net-infograph",
                     "domain_net-clipart", "domain_net-sketch", "cifar100", "mnist", "domain_net-all",
                     "domain_net-real", "google_open_image"],
        "optional_sub_tasks": []
    },
    {
        "task_id": 'a1182934-69d2-4a46-bcbb-feab912cce2c',
        "problem_type": "object_detection", 
        "base_dataset": "vis_drone",
        "adaptation_dataset": "stanford_campus_dataset",
        "base_evaluation_metrics": ["mAP"],
        "adaptation_evaluation_metrics": ["mAP"],
        "whitelist": ["imagenet_1k", "coco2014", "imagenet_bbox", "voc2009", "pool_car_detection", "xview",
                     "domain_net-quickdraw", "domain_net-painting", "domain_net-infograph",
                     "domain_net-clipart", "domain_net-sketch", "cifar100", "mnist", "domain_net-all",
                     "domain_net-real", "google_open_image", "deep_fashion", "deep_fashion_2"],
        "optional_sub_tasks": []
    }
    ,
    {
        "task_id": '7589909f-ca68-4289-8624-25c62c49f0c6',
        "problem_type": "object_detection",
        "base_dataset": "widerperson",
        "adaptation_dataset": "nightowls",
        "base_evaluation_metrics": ["mAP"],
        "adaptation_evaluation_metrics": ["mAP"],
        "whitelist": ["imagenet_1k", "coco2014", "imagenet_bbox", "voc2009", "pool_car_detection", "xview",
             "domain_net-quickdraw", "domain_net-painting", "domain_net-infograph",
             "domain_net-clipart", "domain_net-sketch", "cifar100", "mnist", "domain_net-all",
             "domain_net-real", "google_open_image", "deep_fashion", "deep_fashion_2", "vis_drone", "stanford_campus_dataset"],
        "optional_sub_tasks": []
    },
    {
    "task_id": '7e360ff2-291f-4bbc-b2c5-8a484f04e035',
    "problem_type": "image_classification",
    "base_dataset": "domain_net-painting",
    "base_evaluation_metrics": ["accuracy", "top_5_accuracy", "roc_auc", "cross_entropy_logloss", "brier_score", "weighted_accuracy"],
    "adaptation_dataset": "domain_net-real",
    "adaptation_evaluation_metrics": ["accuracy", "top_5_accuracy", "roc_auc", "cross_entropy_logloss", "brier_score", "weighted_accuracy"],
    "whitelist": ["imagenet_1k", "coco2014", "voc2009", "imagenet_bbox",
                 "domain_net-quickdraw", "domain_net-infograph",
                 "domain_net-clipart", "domain_net-sketch", "cifar100", "domain_net-all",
                 "face_detection", "pool_car_detection", "google_open_image", 
                  "mars_surface_imgs", "msl_curiosity_imgs", "deep_fashion", "deep_fashion_2", "xview", 
                  "stanford_campus_dataset", "vis_drone"],
    "optional_sub_tasks": []
    },
    {
        "task_id": '10c7919a-a8e3-440d-9bf9-959d4f1f880c',
        "problem_type": "object_detection", 
        "base_dataset": "stanford_campus_dataset",
        "adaptation_dataset": "pool_car_detection",
        "base_evaluation_metrics": ["mAP"],
        "adaptation_evaluation_metrics": ["mAP"],
        "whitelist": ["imagenet_1k", "coco2014", "voc2009", "imagenet_bbox",
                     "domain_net-quickdraw", "domain_net-painting", "domain_net-infograph",
                     "domain_net-clipart", "domain_net-sketch", "cifar100", "domain_net-all",
                     "face_detection", "domain_net-real", "google_open_image",
                      "mars_surface_imgs", "msl_curiosity_imgs", "deep_fashion", "deep_fashion_2", 
                      "xview"],
        "optional_sub_tasks": []
    }
    ,
    {
        "task_id": "problem_test_machine_translation",
        "problem_type": "machine_translation",
        "base_dataset": "wikimatrix-fas",
        "adaptation_dataset": "wikimatrix-fas",
        "base_evaluation_metrics": ["bleu"],
        "adaptation_evaluation_metrics": ["bleu"],
        "whitelist": []
    }
    ,
    {
        "task_id": '06023f86-a66b-4b2c-8b8b-951f5edd0f22',
        "problem_type": "machine_translation",
		"base_dataset": "global_voices",
		"base_evaluation_metrics": ["bleu"],
		"adaptation_dataset": "ted_talks",
		"adaptation_evaluation_metrics": ["bleu"],
       "whitelist": []
    }
    ,
    {
        "task_id": '3159d8ef-dfef-48fc-9325-829258be5f28',
        "problem_type": "machine_translation",
        "base_dataset": "cc_aligned_sinhala",
        "base_evaluation_metrics": ["bleu"],
        "adaptation_dataset": "wikimatrix-sin",
        "adaptation_evaluation_metrics": ["bleu"],
       "whitelist": []
    }
    ,
    {
        "task_id": '9877df6a-746f-400e-ac7a-71ca87f50057',
        "problem_type": "machine_translation",
        "base_dataset": "cc_aligned_polish",
        "base_evaluation_metrics": ["bleu"],
        "adaptation_dataset": "wikimatrix-pol",
        "adaptation_evaluation_metrics": ["bleu"],
       "whitelist": []
    }
    ,
    {
        "task_id": '36ff86c2-9403-4fbc-af78-ba2fb54c1734',
        "problem_type": "machine_translation",
        "base_dataset": "cc_aligned_persian",
        "base_evaluation_metrics": ["bleu"],
        "adaptation_dataset": "wikimatrix-fas",
        "adaptation_evaluation_metrics": ["bleu"],
       "whitelist": []
    }
    
]


In [ ]:
# These tasks are not truly used in the evaluation, they are only used to test changes made to this repo when no
# changes have been made to the eval tasks. This is to avoid inadvertent updates to the eval tasks. 
test_eval_tasks = [
       {
        "task_id": "problem_test_image_classification",
        "problem_type": "image_classification",
		"base_dataset": "mnist",
		"base_evaluation_metrics": ["accuracy", "top_5_accuracy", "roc_auc", "cross_entropy_logloss", "brier_score", "weighted_accuracy"],
		"adaptation_dataset": "mnist",
		"adaptation_evaluation_metrics": ["accuracy", "top_5_accuracy", "roc_auc", "cross_entropy_logloss", "brier_score", "weighted_accuracy"],
        "whitelist": ["imagenet_1k", "coco2014", "voc2009", "imagenet_bbox", 
                     "domain_net-quickdraw", "domain_net-painting", "domain_net-infograph",
                     "domain_net-clipart", "domain_net-sketch", "cifar100", "domain_net-all",
                     "face_detection", "domain_net-real", "pool_car_detection", "google_open_image",
                      "mars_surface_imgs", "msl_curiosity_imgs", "deep_fashion", "deep_fashion_2", 
                      "xview", "stanford_campus_dataset", "vis_drone"]
    }
]

In [ ]:
eval_tasks = [
    {
        "task_id": "7260a913-f1fd-4436-88bd-37a49f7d4ef8",
        "problem_type": "image_classification",
		"base_dataset": "hotels_50k-hotels",
		"base_evaluation_metrics": ["accuracy", "top_5_accuracy", "roc_auc", "cross_entropy_logloss", "brier_score", "weighted_accuracy"],
		"adaptation_dataset": "hotels_50k-trafficcam",
		"adaptation_evaluation_metrics": ["accuracy", "top_5_accuracy", "roc_auc", "cross_entropy_logloss", "brier_score", "weighted_accuracy"],
        "whitelist": ["imagenet_1k", "coco2014", "voc2009", "imagenet_bbox", "mnist",
                     "domain_net-quickdraw", "domain_net-painting", "domain_net-infograph",
                     "domain_net-clipart", "domain_net-sketch", "cifar100", "domain_net-all",
                     "face_detection", "domain_net-real", "pool_car_detection", "google_open_image",
                      "mars_surface_imgs", "msl_curiosity_imgs", "deep_fashion", "deep_fashion_2", 
                      "xview", "stanford_campus_dataset", "vis_drone"]
    },
    {
        "task_id": "7f11ce22-981f-4952-94e0-e4be24f686eb",
        "problem_type": "image_classification",
		"base_dataset": "places2-base",
		"base_evaluation_metrics": ["accuracy", "top_5_accuracy", "roc_auc", "cross_entropy_logloss", "brier_score", "weighted_accuracy"],
		"adaptation_dataset": "places2-adpt",
		"adaptation_evaluation_metrics": ["accuracy", "top_5_accuracy", "roc_auc", "cross_entropy_logloss", "brier_score", "weighted_accuracy"],
        "whitelist": ["imagenet_1k", "coco2014", "voc2009", "imagenet_bbox", "mnist",
                     "domain_net-quickdraw", "domain_net-painting", "domain_net-infograph",
                     "domain_net-clipart", "domain_net-sketch", "cifar100", "domain_net-all",
                     "face_detection", "domain_net-real", "pool_car_detection", "google_open_image",
                      "mars_surface_imgs", "msl_curiosity_imgs", "deep_fashion", "deep_fashion_2", 
                      "xview", "stanford_campus_dataset", "vis_drone"]
    },
    {
        "task_id": "b453dc4e-5b8e-42b5-a748-d0e59113ec69",
        "problem_type": "image_classification",
		"base_dataset": "nwpu_resisc45",
		"base_evaluation_metrics": ["accuracy", "top_5_accuracy", "roc_auc", "cross_entropy_logloss", "brier_score", "weighted_accuracy"],
		"adaptation_dataset": "pattern_net",
		"adaptation_evaluation_metrics": ["accuracy", "top_5_accuracy", "roc_auc", "cross_entropy_logloss", "brier_score", "weighted_accuracy"],
        "whitelist": ["imagenet_1k", "coco2014", "voc2009", "imagenet_bbox", "mnist",
                     "domain_net-quickdraw", "domain_net-painting", "domain_net-infograph",
                     "domain_net-clipart", "domain_net-sketch", "cifar100", "domain_net-all",
                     "face_detection", "domain_net-real", "pool_car_detection", "google_open_image",
                      "mars_surface_imgs", "msl_curiosity_imgs", "deep_fashion", "deep_fashion_2", 
                      "xview", "stanford_campus_dataset", "vis_drone"]
    },
    {
        "task_id": "ad348ff9-f6c2-4b21-9bbc-34de6e1ffe10",
        "problem_type": "image_classification",
		"base_dataset": "google_landmarks_v1",
		"base_evaluation_metrics": ["accuracy", "top_5_accuracy", "roc_auc", "cross_entropy_logloss", "brier_score", "weighted_accuracy"],
		"adaptation_dataset": "google_landmarks_v2",
		"adaptation_evaluation_metrics": ["accuracy", "top_5_accuracy", "roc_auc", "cross_entropy_logloss", "brier_score", "weighted_accuracy"],
        "whitelist": ["imagenet_1k", "coco2014", "voc2009", "imagenet_bbox", "mnist",
                     "domain_net-quickdraw", "domain_net-painting", "domain_net-infograph",
                     "domain_net-clipart", "domain_net-sketch", "cifar100", "domain_net-all",
                     "face_detection", "domain_net-real", "pool_car_detection", "google_open_image",
                      "mars_surface_imgs", "msl_curiosity_imgs", "deep_fashion", "deep_fashion_2", 
                      "xview", "stanford_campus_dataset", "vis_drone"]
    },
    {
        "task_id": "3a475004-2ca6-48a4-8569-84bd81116af3",
        "problem_type": "object_detection",
		"base_dataset": "bdd_100k-day",
		"base_evaluation_metrics": ["mAP"],
		"adaptation_dataset": "bdd_100k-night",
		"adaptation_evaluation_metrics": ["mAP"],
        "whitelist": ["imagenet_1k", "coco2014", "voc2009", "imagenet_bbox", "mnist",
                     "domain_net-quickdraw", "domain_net-painting", "domain_net-infograph",
                     "domain_net-clipart", "domain_net-sketch", "cifar100", "domain_net-all",
                     "face_detection", "domain_net-real", "pool_car_detection", "google_open_image",
                      "mars_surface_imgs", "msl_curiosity_imgs", "deep_fashion", "deep_fashion_2", 
                      "xview"]
    },
    {
        "task_id": "0f0706dc-5dc2-4e7d-a1d1-5cbc42344a0a",
        "problem_type": "object_detection",
		"base_dataset": "bdd_100k-all",
		"base_evaluation_metrics": ["mAP"],
		"adaptation_dataset": "idd",
		"adaptation_evaluation_metrics": ["mAP"],
        "whitelist": ["imagenet_1k", "coco2014", "voc2009", "imagenet_bbox", "mnist",
                     "domain_net-quickdraw", "domain_net-painting", "domain_net-infograph",
                     "domain_net-clipart", "domain_net-sketch", "cifar100", "domain_net-all",
                     "face_detection", "domain_net-real", "pool_car_detection", "google_open_image",
                      "mars_surface_imgs", "msl_curiosity_imgs", "deep_fashion", "deep_fashion_2", 
                      "xview"]
    },
    {
        "task_id": "f892c2f4-9184-409c-925a-4ce4f639c5c3",
        "problem_type": "object_detection",
		"base_dataset": "ug2-glider",
		"base_evaluation_metrics": ["mAP"],
		"adaptation_dataset": "ug2-uav",
		"adaptation_evaluation_metrics": ["mAP"],
        "whitelist": ["imagenet_1k", "coco2014", "voc2009", "imagenet_bbox", "mnist",
                     "domain_net-quickdraw", "domain_net-painting", "domain_net-infograph",
                     "domain_net-clipart", "domain_net-sketch", "cifar100", "domain_net-all",
                     "face_detection", "domain_net-real", "pool_car_detection", "google_open_image",
                      "mars_surface_imgs", "msl_curiosity_imgs", "deep_fashion", "deep_fashion_2", 
                      "xview"]
    },
    {
        "task_id": "93eb7ebc-a0da-4a7c-a544-ed8bfb3668bf",
        "problem_type": "object_detection",
		"base_dataset": "auair",
		"base_evaluation_metrics": ["mAP"],
		"adaptation_dataset": "wilddash2",
		"adaptation_evaluation_metrics": ["mAP"],
        "whitelist": ["imagenet_1k", "coco2014", "voc2009", "imagenet_bbox", "mnist",
                     "domain_net-quickdraw", "domain_net-painting", "domain_net-infograph",
                     "domain_net-clipart", "domain_net-sketch", "cifar100", "domain_net-all",
                     "face_detection", "domain_net-real", "pool_car_detection", "google_open_image",
                      "mars_surface_imgs", "msl_curiosity_imgs", "deep_fashion", "deep_fashion_2", 
                      "xview"]
    },
    {
        "task_id": "bbcdae4e-7a55-4240-8e3c-daa5968a77cd",
        "problem_type": "object_detection",
		"base_dataset": "kal_base",
		"base_evaluation_metrics": ["mAP"],
		"adaptation_dataset": "kal_adpt",
		"adaptation_evaluation_metrics": ["mAP"],
        "whitelist": ["imagenet_1k", "coco2014", "imagenet_bbox", "voc2009", "pool_car_detection", 
                      "xview","domain_net-quickdraw", "domain_net-painting", "domain_net-infograph",
                      "domain_net-clipart", "domain_net-sketch", "cifar100", "mnist", "domain_net-all",
                      "domain_net-real", "google_open_image", "deep_fashion", "deep_fashion_2", "vis_drone", 
                      "stanford_campus_dataset", "cowc", "dota", "fair1m", "imagenet_22k"]
    },
    {
        "task_id": "eab7e083-880e-44ba-a412-afe8065e7394",
        "problem_type": "object_detection",
		"base_dataset": "kal_base",
		"base_evaluation_metrics": ["mAP"],
		"adaptation_dataset": "kal_adpt",
		"adaptation_evaluation_metrics": ["mAP"],
        "whitelist": ["imagenet_1k", "coco2014", "imagenet_bbox", "voc2009", "pool_car_detection", 
                      "xview","domain_net-quickdraw", "domain_net-painting", "domain_net-infograph",
                      "domain_net-clipart", "domain_net-sketch", "cifar100", "mnist", "domain_net-all",
                      "domain_net-real", "google_open_image", "deep_fashion", "deep_fashion_2", "vis_drone", 
                      "stanford_campus_dataset", "cowc", "dota", "fair1m", "imagenet_22k"]
    },
    {
        "task_id": 'e145bcd7-8d64-4817-8db1-6307f197449d',
        "problem_type": "machine_translation",
		"base_dataset": "gale",
		"base_evaluation_metrics": ["bleu"],
		"adaptation_dataset": "ted_talks",
		"adaptation_evaluation_metrics": ["bleu"],
        "whitelist": []
    },
    {
        "task_id": '75fb4978-66ce-479e-8ebc-d53ba2f08e57',
        "problem_type": "machine_translation",
		"base_dataset": "paracrawl_portuguese",
		"base_evaluation_metrics": ["bleu"],
		"adaptation_dataset": "pubmed_portuguese",
		"adaptation_evaluation_metrics": ["bleu"],
        "whitelist": []
    },
    {
        "task_id": '7109e533-beb7-4e61-95ef-0cf11cbe3a77',
        "problem_type": "machine_translation",
		"base_dataset": "gale_arabic",
		"base_evaluation_metrics": ["bleu"],
		"adaptation_dataset": "bolt_arabic",
		"adaptation_evaluation_metrics": ["bleu"],
        "whitelist": []
    },
    {
        "task_id": 'a4d0ef87-1970-4b50-a3c1-bc4e98d7caf0',
        "problem_type": "machine_translation",
		"base_dataset": "hunglish",
		"base_evaluation_metrics": ["bleu"],
		"adaptation_dataset": "paracrawl_hungarian",
		"adaptation_evaluation_metrics": ["bleu"],
        "whitelist": []
    },
        {
        "task_id": 'b9a612a8-c0e4-4079-acd7-14a8f3022468',
        "problem_type": "video_classification",
        "base_dataset": "ek_base",
        "adaptation_dataset": "ek_adpt",
        "base_evaluation_metrics": ["accuracy", "top_5_accuracy", "roc_auc", "cross_entropy_logloss", 
                                          "brier_score", "weighted_accuracy"],
        "adaptation_evaluation_metrics": ["accuracy", "top_5_accuracy", "roc_auc", "cross_entropy_logloss", 
                                          "brier_score", "weighted_accuracy"],
        "whitelist": ["imagenet_1k", "Kinetics400", "Kinetics600", "Kinetics700", "UCF101", "ActivityNet", 
                      "Moments-in-Time", "Sports1M", "YouTube8M", "HACS", "HVU", "AVA", 
                      "AVA-kinetics", "charades"]
    }
]

## Add the budgets to the tasks
Calling `add_budget_labels_to_task` adds the budgets in place. Pass `sample=True` only for dev and staging tasks to create budgets for the sample size as well. Evaluation tasks have no sample data. 

In [ ]:
add_budget_labels_to_task(dev_tasks, sample=True)
dev_tasks[0]

In [ ]:
add_budget_labels_to_task(staging_tasks, sample=True)
staging_tasks[0]

In [ ]:
add_budget_labels_to_task(test_eval_tasks, sample=True)
test_eval_tasks[0]

In [ ]:
add_budget_labels_to_task(eval_tasks, sample=True)
eval_tasks[0]

# Create task in firebase

In [ ]:
def create_or_update_task(task, level='dev'):
    """
    Checks if the task exists. If it does, it updates all the fields in
    the task. Otherwise, it creates the task
    """
    _id = task['task_id']
    doc_ref = fb_store.collection(f'{level}_Task').document(_id)
    doc = doc_ref.get()
    if doc.exists:
        print(f"Found task: {_id}. Updating contents")
        for field, value in task.items():
            doc_ref.update({field: value})
    else:
        print(f"Creating new task: {_id}")
        fb_store.collection(f'{level}_Task').document(_id).set(task)

### Create/Update dev tasks

In [ ]:
for task in dev_tasks:
    create_or_update_task(task, level='dev')

### Create/Update staging tasks

In [ ]:
for task in staging_tasks:
    create_or_update_task(task, level='staging')

### Create/Update eval tasks

In [ ]:
for task in test_eval_tasks:
    create_or_update_task(task, level='prod')

In [ ]:
for task in eval_tasks:
    create_or_update_task(task, level='prod')

## Create seed labels for each dataset in the tasks
First, we loop through the tasks and get a set of all the image_classification, object_detection, and video_classification dataset names. There is no concept of seeds for machine translation. We separate the types out because they are handled differently. We do this first because some tasks use the same dataset, and we don't want to compute seed labels for the same set multiple times. Then we will assign the appropriate seed labels to the task. 


This is done after we create the task in firebase because some of the seed label sets are quite large and would result in too large of a request, so the base and adaptation labels are added separately using an update. 

In [ ]:
def get_dataset_names(task_list: list):
    """
    Gets a set of all the dataset names in a task list:
    Args:
        task_list: list of dicts: dev, staging, or eval task list
    """
    cls_datasets, obj_det_datasets, video_datasets = set(), set(), set()
    for task in task_list:
        if task['problem_type'] == "image_classification":
            cls_datasets.add(task['base_dataset'])
            cls_datasets.add(task['adaptation_dataset'])
        elif task['problem_type'] == "object_detection":
            obj_det_datasets.add(task['base_dataset'])
            obj_det_datasets.add(task['adaptation_dataset'])
        elif task['problem_type'] == 'video_classification':
            video_datasets.add(task['base_dataset'])
            video_datasets.add(task['adaptation_dataset'])
    return cls_datasets, obj_det_datasets, video_datasets

In [ ]:
dev_cls_datasets, dev_obj_datasets, dev_vid_datasets = get_dataset_names(dev_tasks)

In [ ]:
staging_cls_datasets, staging_obj_datasets, staging_vid_datasets = get_dataset_names(staging_tasks)

In [ ]:
test_eval_cls_datasets, test_eval_obj_datasets, test_eval_vid_datasets = get_dataset_names(test_eval_tasks)

In [ ]:
eval_cls_datasets, eval_obj_datasets, eval_vid_datasets = get_dataset_names(eval_tasks)

In [ ]:
dev_cls_datasets

In [ ]:
dev_obj_datasets

In [ ]:
dev_vid_datasets

Create a seed generator object that will connect to S3 to get labels. The same object can be used for every dataset. For the development and staging tasks, we use the sample dataset to create the seeds, so both the sample and full dataset will have the same seeds. For the eval tasks, there are no sample datasets, and we create seeds from the full dataset. 

In [ ]:
seed_generator = SeedLabelGenerator()

In [ ]:
def get_seeds(cls_datasets, obj_det_datasets, video_datasets, sample, level):
    seeds = {}
    for dataset_name in cls_datasets:
        print(dataset_name)
        lbls = seed_generator.get_train_labels(dataset_name=dataset_name, level=level, sample=sample)
        num_classes = lbls['class'].nunique()
        print(f"num unique classes in lbls: {num_classes}")
        try:
              seeds[dataset_name] = seed_generator.create_seed_lbls(lbls, problem_type='image_classification')
        except Exception as e:
              print(e)
    
    for dataset_name in obj_det_datasets:
        if dataset_name == "xview" or dataset_name == "pool_car_detection":
            print(f"Setting {dataset_name} seeds to None.")
            seeds[dataset_name] = {
                "0": None,
                "1": None,
                "2": None,
                "3": None
            }
        else:
            print(dataset_name)
            lbls = seed_generator.get_train_labels(dataset_name=dataset_name, level=level, sample=sample)
            num_ids = lbls['id'].nunique()
            try:
                seeds[dataset_name] = seed_generator.create_seed_lbls(lbls, problem_type='object_detection')

            except Exception as e:
                print(f"Error, could not generate seed labels: {e}")
    
    for dataset_name in video_datasets:
        print(dataset_name)
        lbls = seed_generator.get_train_labels(dataset_name=dataset_name, level=level, sample=sample)
        try:
            seeds[dataset_name] = seed_generator.create_seed_lbls(lbls, problem_type='video_classification')

        except Exception as e:
            print(e)
    return seeds

In [ ]:
# For dev/staging tasks, use the labels from the sample size feather file so that the full and sample size have 
# the same sample seeds.
print("Getting dev sample seeds")
dev_seeds = get_seeds(dev_cls_datasets, dev_obj_datasets, dev_vid_datasets, False, level='dev')

In [ ]:
print("Getting staging seeds")
staging_seeds = get_seeds(staging_cls_datasets, staging_obj_datasets, staging_vid_datasets, False, level='staging')

In [ ]:
print("Getting test_eval seeds")
# The level is set to dev because the test problems are not in the lwll-datasets bucket,
# but not lwll-eval-datasets bucket
test_eval_seeds = get_seeds(test_eval_cls_datasets, test_eval_obj_datasets, test_eval_vid_datasets, False, level='eval')

In [ ]:
print("Getting eval seeds")
eval_seeds = get_seeds(eval_cls_datasets, eval_obj_datasets, eval_vid_datasets, False, level='eval')

### Store the seeds

In [ ]:
def save_seeds(task, all_seeds, level='dev'):
    if level not in ['dev', 'staging', 'prod']:
        raise Exception(f"Expected level to be `dev`, `staging`, or `prod`, but got {level}")
    if task['problem_type'] != "machine_translation":
        _id = task['task_id']
        print(_id)
        task_ref = fb_store.collection(f'{level}_Task').document(task['task_id'])
        try:
            for key, seeds in all_seeds[task['base_dataset']].items():
                task_ref.update({f"base_seed_labels.{key}": seeds})
        except Exception as e:
            print(e)
            dataset = task['base_dataset']
            print(f"failed on {dataset} at checkpoint {key}") 
        try:
            for key, seeds in all_seeds[task['adaptation_dataset']].items():
                task_ref.update({f"adaptation_seed_labels.{key}": seeds})
        except Exception as e:
            print(e)
            dataset = task['adaptation_dataset']
            print(f"failed on {dataset} at checkpoint {key}")

In [ ]:
for task in dev_tasks:
    if task['problem_type'] != "machine_translation":
        _id = task['task_id']
        print(f"getting seeds for task {_id}")
        save_seeds(task, dev_seeds, 'dev')

In [ ]:
for task in staging_tasks:
    if task['problem_type'] != "machine_translation":
        _id = task['task_id']
        print(f"saving seeds for task {_id}")
        save_seeds(task, staging_seeds, 'staging')

In [ ]:
for task in test_eval_tasks:
    if task['problem_type'] != "machine_translation":
        _id = task['task_id']
        print(f"getting seeds for task {_id}")
        save_seeds(task, test_eval_seeds, 'prod')

In [ ]:
for task in eval_tasks:
    if task['problem_type'] != "machine_translation":
        _id = task['task_id']
        print(f"getting seeds for task {_id}")
        save_seeds(task, eval_seeds, 'prod')

## Add class overlap 
For the tasks, we get the % overlap in class names from base -> adaptation and adaptation -> base so that performers can decide if they want to do unsupervised domain adaptation

In [ ]:
def add_class_overlap(task: dict, level: str):
    """
    Retrieves class names for base and adaptation datasets, gets dataset metadata from FB, calculates % overlap.
    Args:
        task: dict in format
        {
            "task_id": "prod_test_image_classification",
            "problem_type": "image_classification",
            "base_dataset": "mnist",
            "base_evaluation_metrics": ["accuracy"],
            "adaptation_dataset": "mnist",
            "adaptation_evaluation_metrics": ["accuracy"],
            "whitelist": ["imagenet_1k", "coco2014", "voc2009", "imagenet_bbox",
                         "domain_net-quickdraw", "domain_net-painting", "domain_net-infograph",
                         "domain_net-clipart", "domain_net-sketch", "cifar100", "domain_net-all",
                         "face_detection", "domain_net-real", "pool_car_detection", "google_open_image"]
        }
        level: str, valid values are 'dev', 'staging', 'prod'

    """
    if level not in ["dev", "staging", "prod"]:
        print("Invalid task type")
        return
    if task['problem_type'] == "machine_translation":
        print("Skipping machine translation problem")
        return
    
    # get base classes
    base_meta = fb_store.collection('DatasetMetadata').document(task['base_dataset']).get().to_dict()
    base_classes = base_meta['classes']
    base_classes = [str(item).lower() for item in base_classes]
    
    # get adapt classes
    adaptation_meta = fb_store.collection('DatasetMetadata').document(task['adaptation_dataset']).get().to_dict()
    adaptation_classes = adaptation_meta['classes']
    adaptation_classes = [str(item).lower() for item in adaptation_classes]
    
    # calculate overlap and store
    intersection = len(set(base_classes).intersection(set(adaptation_classes)))
    base_to_adapt_overlap = intersection / len(base_classes) 
    if base_to_adapt_overlap == 0:
        base_to_adapt_overlap = 0.0001
    adapt_to_base_overlap = intersection / len(adaptation_classes)
    if adapt_to_base_overlap == 0:
        adapt_to_base_overlap = 0.0001
    if task["problem_type"] == "image_classification":
        base = task["base_dataset"]
        adapt = task["adaptation_dataset"]
    task_ref = fb_store.collection(f'{level}_Task').document(task['task_id'])
    task_ref.update({"uda_base_to_adapt_overlap_ratio": base_to_adapt_overlap})
    task_ref.update({"uda_adapt_to_base_overlap_ratio": adapt_to_base_overlap})

In [ ]:
for task in dev_tasks:
    _id = task['task_id']
    print(f"adding uda overlap to task {_id}")  
    add_class_overlap(task, level="dev")

In [ ]:
for task in staging_tasks:
    _id = task['task_id']
    print(f"adding uda overlap to task {_id}")  
    add_class_overlap(task, level="staging")

In [ ]:
for task in test_eval_tasks:
    _id = task['task_id']
    print(f"adding uda overlap to task {_id}")  
    add_class_overlap(task, level="prod")

In [ ]:
for task in eval_tasks:
    _id = task['task_id']
    print(f"adding uda overlap to task {_id}")  
    add_class_overlap(task, level="prod")

## Update any field

In [ ]:
def update_task_field(task_id: str, level: str, field: str, value: Union[list, str]):
    """
    Updates any field of a task
    Args:
        task_id: str, The unique id of the task
        level: str, valid values are 'dev', 'staging', 'prod'
        field: str, the key of the field to be updated
        value: Union[str, list], the data to update
    """
    if level not in ["dev", "staging", "prod"]:
        print("Invalid task type")
        return    

    # Get the task
    task_ref = fb_store.collection(f'{level}_Task').document(task_id)
    
    # update the task
    try:
        task_ref.update({field: value})
    except KeyError:
        task_ref.create({field: value})

In [ ]:
for task in staging_tasks:
    task_id = task['task_id']
    if 'optional_sub_tasks' in task.keys():
        print(f"updating task {task_id}")
        update_task_field(task_id, 'staging', 'optional_sub_tasks', task['optional_sub_tasks'])


In [ ]:
for task in eval_tasks:
    task_id = task['task_id']
    update_task_field(task_id, 'prod', 'base_label_budget_sample', task['base_label_budget_sample'])
    update_task_field(task_id, 'prod', 'adaptation_label_budget_sample', task['adaptation_label_budget_sample'])

In [ ]:
for task in eval_tasks:
    task_id = task['task_id']
    update_task_field(task_id, 'prod', 'base_label_budget_sample', task['base_label_budget_sample'])
    update_task_field(task_id, 'prod', 'base_label_budget_full', task['base_label_budget_full'])
    update_task_field(task_id, 'prod', 'adaptation_label_budget_sample', task['adaptation_label_budget_sample'])
    update_task_field(task_id, 'prod', 'adaptation_label_budget_full', task['adaptation_label_budget_full'])

In [ ]:
task_ref = fb_store.collection(f'dev_Task').document('problem_test_image_classification')

In [ ]:
doc = task_ref.get()

In [ ]:
doc.to_dict()

## Removing Data

In [ ]:
# active key from bool to str
# def rename_active_key(coll_ref, batch_size):
#     docs = coll_ref.get()
#     for doc in docs:
#         data = doc.to_dict()
#         if data['active'] is True:
#             data['active'] = 'In Progress'
#         elif data['active'] is False:
#             data['active'] = 'Complete'
#         coll_ref.document(doc.id).update(data)

In [ ]:
# rename_active_key(fb_store.collection('dev_Session'))

In [ ]:
# def remove_data_urls(coll_ref):
#     docs = coll_ref.get()
#     for doc in docs:
#         data = doc.to_dict()
#         data['full_data_url'] = None
#         data['sample_data_url'] = None
#         coll_ref.document(doc.id).update(data)

In [ ]:
# remove_data_urls(fb_store.collection('DatasetMetadata'))

In [ ]:
# Purge all Sessions WARNING YOU USUALLY DO NOT WANT TO DO THIS. 
# THIS IS ONLY USEFUL IN EARLY DEVELOPMENT WHILE DATA SCHEMAS ARE CHANGING

# def delete_collection(coll_ref, batch_size):
#     docs = coll_ref.limit(batch_size).get()
#     deleted = 0

#     for doc in docs:
#         print(u'Deleting doc {} => {}'.format(doc.id, doc.to_dict()))
#         doc.reference.delete()
#         deleted = deleted + 1

#     if deleted >= batch_size:
#         return delete_collection(coll_ref, batch_size)

In [ ]:
delete_collection(fb_store.collection('dev_Session'), 20)